# Using Gambit with OpenSpiel

This tutorial demonstrates the interoperability of the Gambit and OpenSpiel Python packages for game-theoretic analysis.

Gambit provides a range of methods to compute exact and close approximations of equilibria for games. OpenSpiel provides a variety of iterative multi-agent learning algorithms, which may or may not converge to equilibria.

Another key distinction is that the PyGambit API allows the user a simple way to define custom games (see tutorials 1-3). This is also possible in OpenSpiel for normal-form games, and you can load `.efg` files created from Gambit for the extensive-form, however some of the key functionality for iterated learning of strategies is only available for games from the built-in library (see the [OpenSpiel documentation](https://openspiel.readthedocs.io/en/latest/games.html)).

This tutorial demonstrates:

1. Transferring examples of normal (strategic) form and extensive-form games between OpenSpiel and Gambit
2. Simulating evolutionary dynamics of populations of strategies in OpenSpiel for normal-form games
3. Training agents using self-play of extensive-form games in OpenSpiel to create strategies
4. Comparing the strategies from OpenSpiel against equilibrium strategies computed with Gambit

Note: The OpenSpiel code was adapted from the introductory tutorial for the OpenSpiel API on colab [here](https://colab.research.google.com/github/deepmind/open_spiel/blob/master/open_spiel/colabs/OpenSpielTutorial.ipynb).

In [ ]:
from io import StringIO

import matplotlib.pyplot as plt
import numpy as np
import pyspiel
from open_spiel.python import rl_environment
from open_spiel.python.algorithms import tabular_qlearner
from open_spiel.python.algorithms.gambit import export_gambit
from open_spiel.python.egt import dynamics
from open_spiel.python.egt.utils import game_payoffs_array

import pygambit as gbt

## OpenSpiel game library

OpenSpiel has a large selection of games available in its [library](https://openspiel.readthedocs.io/en/latest/games.html). Many of these will not be amenable to equilibrium computation with Gambit, due to their size. For the purposes of this tutorial, we'll pick some of the smallest games from the list below.

In [ ]:
print(pyspiel.registered_names())

## Normal-form games from the OpenSpiel library

Let's start with the simple normal-form game of rock-paper-scissors, in which the payoffs can be represented by a 3x3 matrix.

Load matrix rock-paper-scissors from OpenSpiel:

In [ ]:
ops_matrix_rps_game = pyspiel.load_game("matrix_rps")

In order to simulate a playthrough of the game, you can first initialise a game state:

In [ ]:
state = ops_matrix_rps_game.new_initial_state()
state

The possible actions for both players (player 0 and player 1) are Rock, Paper and Scissors, but these are not labelled and must be accessed via integer indices:

In [ ]:
print(state.legal_actions(0)) # Player 0 (row) actions
print(state.legal_actions(1)) # Player 1 (column) actions

Since Rock-paper-scissors is a 1-step simultaneous-move normal-form game, we'll apply a list of player actions in one step to reach the terminal state.

Let's simulate player 0 playing Rock (0) and player 1 playing Paper (1):

In [ ]:
state.apply_actions([0, 1])
state

OpenSpiel can generate an NFG representation of the game loadable in Gambit:

In [ ]:
nfg_matrix_rps_game = pyspiel.game_to_nfg_string(ops_matrix_rps_game)
nfg_matrix_rps_game

Now let's load the NFG in Gambit. Since Gambit's `read_nfg` function expects a file like object, we'll convert the string with `io.StringIO`.
We can also add labels for the actions to make the output more interpretable:

In [ ]:
gbt_matrix_rps_game = gbt.read_nfg(StringIO(nfg_matrix_rps_game))

gbt_matrix_rps_game.title = "Rock-Paper-Scissors"

for player in gbt_matrix_rps_game.players:
    player.strategies[0].label = "Rock"
    player.strategies[1].label = "Paper"
    player.strategies[2].label = "Scissors"

gbt_matrix_rps_game

The unique equilibrium mixed strategy profile for both players is to choose rock, paper, and scissors with equal probability:

In [ ]:
gbt.nash.lcp_solve(gbt_matrix_rps_game).equilibria[0]

We can use OpenSpiel's dynamics module to demonstrate evolutionary game theory dynamics, or "replicator dynamics", which models how a mixed strategy profile evolves over time based on how the strategies (e.g., choice of actions A, B, C with probabilities X, Y, Z) perform against one another.

Let's start with an initial profile that is not at equilibrium, but weighted towards scissors with proportions: 30% Rock, 30% Paper, 40% Scissors:

In [ ]:
matrix_rps_payoffs = game_payoffs_array(ops_matrix_rps_game)
dyn = dynamics.SinglePopulationDynamics(matrix_rps_payoffs, dynamics.replicator)
x = np.array([0.3, 0.3, 0.4])
dyn(x)

`dyn(x)` calculates the rate of change (derivative) for each strategy in the current profile and returns how fast each strategy's frequency is changing.

In replicator dynamics, a pure strategy that performs well against others will increase in frequency, while strategies performing worse will decrease.
In our rock-paper-scissors example, the performance of each pure strategy (action) depends on the probability it is assigned in the mixed strategy profile. At the start, whilst there are more players choosing scissors as their action, then rock will perform well and increase in frequency (be more likely to get played in subsequent rounds), while paper will perform poorly and decrease in frequency. We can plot how the frequency of each strategy changes over time:

In [ ]:
def plot_rps_dynamics(proportions, steps=100, alpha=0.1, plot_average_strategy=False):
    x = np.array(proportions)
    rock_proportions = [x[0]]
    paper_proportions = [x[1]]
    scissors_proportions = [x[2]]
    y = []
    for _ in range(steps):
        x += alpha * dyn(x)
        rock_proportions.append(x[0])
        paper_proportions.append(x[1])
        scissors_proportions.append(x[2])
        if plot_average_strategy:
            y.append([np.mean(rock_proportions),
                      np.mean(paper_proportions),
                      np.mean(scissors_proportions)
                      ])
        else:
            y.append(x.copy())
    y = np.array(y)

    plt.plot(y[:, 0], label="Rock")
    plt.plot(y[:, 1], label="Paper")
    plt.plot(y[:, 2], label="Scissors")
    plt.xlabel("Time step")
    if plot_average_strategy:
        plt.ylabel("Strategy frequency average up to time step")
    else:
        plt.ylabel("Strategy frequency")
    plt.legend()
    plt.show()

plot_rps_dynamics([0.3, 0.3, 0.4])

Through the dynamics, we can see that the population proportions oscillate around the equilibrium point (1/3, 1/3, 1/3) without converging to it, because the best strategy depends on the likelihood of the opponents' actions, as defined by the current action probabilities.

However, if we start with the initial population already at the equilibrium mixed strategy profile computed by Gambit (each action is chosen exactly 1/3 of the time), the strategy frequencies will remain constant over time (at the equilibrium point):

In [ ]:
plot_rps_dynamics([1/3, 1/3, 1/3])

When starting from an unbalanced initial mixed strategy profile, the strategy frequencies will oscillate around the equilibrium point without converging to it. However, if we plot the average strategy frequencies over time, we can see that this begins to converge to the equilibrium point:

In [ ]:
plot_rps_dynamics([0.3, 0.3, 0.4], plot_average_strategy=True)

## Normal-form games created with Gambit

You can also set up a normal-form game in Gambit and export it to OpenSpiel. Here we demonstrate this with the simple Prisoner's Dilemma game:

In [ ]:
player1_payoffs = np.array([[-1, -3], [0, -2]])
player2_payoffs = np.transpose(player1_payoffs)

gbt_prisoners_dilemma_game = gbt.Game.from_arrays(
    player1_payoffs,
    player2_payoffs,
    title="Prisoner's Dilemma"
)
gbt_prisoners_dilemma_game

In [ ]:
gbt.nash.lcp_solve(gbt_prisoners_dilemma_game).equilibria[0]

As expected, Gambit computes the unique equilibrium strategy for both players as choosing cooperate with probability 0 and defect with probability 1.

To re-create the game in OpenSpiel we extract the player payoffs to NumPy arrays, which are then used to create a matrix game in OpenSpiel:

In [ ]:
p1_payoffs, p2_payoffs = gbt_prisoners_dilemma_game.to_arrays(dtype=float)
ops_prisoners_dilemma_game = pyspiel.create_matrix_game(
    gbt_prisoners_dilemma_game.title,
    "Classic Prisoner's Dilemma",  # description
    [strategy.label for strategy in gbt_prisoners_dilemma_game.players[0].strategies],
    [strategy.label for strategy in gbt_prisoners_dilemma_game.players[1].strategies],
    p1_payoffs,
    p2_payoffs
)

Like rock-paper-scissors, the Prisoner's Dilemma is a 1-step simultaneous-move normal-form game; we'll apply a list of player actions in one step to reach the terminal state. Let's have both player choose to defect (1):

In [ ]:
state = ops_prisoners_dilemma_game.new_initial_state()
state.apply_actions([1, 1])
state

Unlike in rock-paper-scissors, the Prisoner's Dilemma has a dominant strategy equilibrium, in which both players defect.
Using evolutionary dynamics, we can see that a population starting with a mix of cooperators and defectors will evolve towards all defectors over time:

In [ ]:
matrix_pd_payoffs = game_payoffs_array(ops_prisoners_dilemma_game)
pd_dyn = dynamics.SinglePopulationDynamics(matrix_pd_payoffs, dynamics.replicator)

def plot_pd_dynamics(proportions, steps=100, alpha=0.1):
    x = np.array(proportions)
    y = []
    for _ in range(steps):
        x += alpha * pd_dyn(x)
        y.append(x.copy())
    y = np.array(y)
    plt.plot(y[:, 0], label="Cooperate")
    plt.plot(y[:, 1], label="Defect")
    plt.xlabel("Time step")
    plt.ylabel("Frequency")
    plt.legend()
    plt.show()

plot_pd_dynamics([0.8, 0.2])

<!-- ## Extensive-form example: Silly1111 Poker -->

<!-- Silly poker is a variant imperfect information one-card poker game introduced in tutorial 3, but in which there are 3 possible cards (J, Q, K) instead of 2. -->

## Extensive-form games from the OpenSpiel library

For extensive-form games, OpenSpiel can export to the EFG format used by Gambit. Here we demonstrate this with **Tiny Hanabi**, loaded from the OpenSpiel [game library](https://openspiel.readthedocs.io/en/latest/games.html):

In [ ]:
ops_hanabi_game = pyspiel.load_game("tiny_hanabi")
efg_hanabi_game = export_gambit(ops_hanabi_game)
efg_hanabi_game

Now let's load the EFG in Gambit.
We can then compute equilibria strategies for the players as usual:

In [ ]:
gbt_hanabi_game = gbt.read_efg(StringIO(efg_hanabi_game))
eqm = gbt.nash.lcp_solve(gbt_hanabi_game).equilibria[0]

In [ ]:
from draw_tree import draw_tree

draw_tree(
    gbt_hanabi_game,
    color_scheme="gambit",
    edge_thickness=2,
    action_label_position=0.8,
    shared_terminal_depth=True
)

We can look at player 0's equilibrium strategy:

In [ ]:
eqm["Pl0"]

...and use Gambit to explore what those numbers actually mean for player 0:

In [ ]:
for infoset, mixed_action in eqm["Pl0"].mixed_actions():
    print(
        f"At information set {infoset.number}, "
        f"Player 0 plays action 0 with probability: {mixed_action['p0a0']}"
        f" and action 1 with probability: {mixed_action['p0a1']}"
        f" and action 2 with probability: {mixed_action['p0a2']}"
    )

For player 1, we can do the same:

In [ ]:
eqm["Pl1"]

In [ ]:
for infoset, mixed_action in eqm["Pl1"].mixed_actions():
    print(
        f"At information set {infoset.number}, "
        f"Player 1 plays action 0 with probability: {mixed_action['p1a0']}"
        f" and action 1 with probability: {mixed_action['p1a1']}"
        f" and action 2 with probability: {mixed_action['p1a2']}"
    )

Let's now train 2 agents using independent Q-learning on Tiny Hanabi, and play them against eachother.

We can compare the learned strategies played to the equilibrium strategies computed by Gambit.

First let's open the RL environment for Tiny Hanabi and create the agents, one for each player (2 players in this case):

In [ ]:
# Create the environment
env = rl_environment.Environment("tiny_hanabi")
num_players = env.num_players
num_actions = env.action_spec()["num_actions"]

# Create the agents
agents = [
    tabular_qlearner.QLearner(player_id=idx, num_actions=num_actions)
    for idx in range(num_players)
]

Now we can train the Q-learning agents in self-play:

In [ ]:
for cur_episode in range(30000):
  if cur_episode % 10000 == 0:
    print(f"Episodes: {cur_episode}")

  time_step = env.reset()
  while not time_step.last():
    player_id = time_step.observations["current_player"]
    agent_output = agents[player_id].step(time_step)
    time_step = env.step([agent_output.action])

  # Episode is over, step all agents with final info state.
  for agent in agents:
    agent.step(time_step)

print(f"Episodes: {cur_episode+1}")

Let's check out the strategies our agents have learned by playing them against eachother again, this time in evaluation mode (setting `is_evaluation=True`):

In [ ]:
time_step = env.reset()

while not time_step.last():
  print("")
  print(env.get_state)

  player_id = time_step.observations["current_player"]
  agent_output = agents[player_id].step(time_step, is_evaluation=True)
  print(f"Agent {player_id} chooses {env.get_state.action_to_string(agent_output.action)}")
  time_step = env.step([agent_output.action])

print("")
print(env.get_state)
print(f"Rewards: {time_step.rewards}")

Are the learned strategies chosen by p0 and p1 consistent with an equilibrium computed by Gambit?

When I ran the above I got the final game state `p0:d0 p1:d0 p0:a2 p1:a0` with payoffs `[10.0, 10.0]`. This is consistent with the equilibrium computed by Gambit:
- The node `p0:d0 p1:d0` is part of player 0's information set 0.
- p0 picks a2 which matches the first equilibrium strategy in `eqm['Pl0']` where action `p0a2` is played with probability 1.0.
- This puts player 1 in their information set 2, and player 1 picks action 0, which is consistent with `eqm['Pl1']` where action `p1a0` is played with probability 1.0.

## Extensive-form games created with Gambit

It's also possible to create an extensive-form game in Gambit and export it to OpenSpiel. Here we demonstrate this with the one-card poker game introduced in tutorial 3:

In [ ]:
gbt_one_card_poker = gbt.Game.new_tree(
    players=["Alice", "Bob"],
    title="Stripped-Down Poker: a simple game of one-card poker from Reiley et al (2008)."
)

gbt_one_card_poker.append_move(
    gbt_one_card_poker.root,
    player=gbt_one_card_poker.players.chance,
    actions=["King", "Queen"]  # By default, chance actions have equal probabilities
)

for node in gbt_one_card_poker.root.children:
    gbt_one_card_poker.append_move(
        node,
        player="Alice",
        actions=["Bet", "Fold"]
    )

gbt_one_card_poker.append_move(
    [
        gbt_one_card_poker.root.children["King"].children["Bet"],
        gbt_one_card_poker.root.children["Queen"].children["Bet"]
    ],
    player="Bob",
    actions=["Call", "Fold"]
)

win_big = gbt_one_card_poker.add_outcome([2, -2], label="Win Big")
win = gbt_one_card_poker.add_outcome([1, -1], label="Win")
lose_big = gbt_one_card_poker.add_outcome([-2, 2], label="Lose Big")
lose = gbt_one_card_poker.add_outcome([-1, 1], label="Lose")

# Alice folds, Bob wins small
gbt_one_card_poker.set_outcome(
    gbt_one_card_poker.root.children["King"].children["Fold"],
    lose
)
gbt_one_card_poker.set_outcome(
    gbt_one_card_poker.root.children["Queen"].children["Fold"],
    lose
)

# Bob sees Alice Bet and calls, correctly believing she is bluffing, Bob wins big
gbt_one_card_poker.set_outcome(
    gbt_one_card_poker.root.children["Queen"].children["Bet"].children["Call"],
    lose_big
)

# Bob sees Alice Bet and calls, incorrectly believing she is bluffing, Alice wins big
gbt_one_card_poker.set_outcome(
    gbt_one_card_poker.root.children["King"].children["Bet"].children["Call"],
    win_big
)

# Bob does not call Alice's Bet, Alice wins small
gbt_one_card_poker.set_outcome(
    gbt_one_card_poker.root.children["King"].children["Bet"].children["Fold"],
    win
)
gbt_one_card_poker.set_outcome(
    gbt_one_card_poker.root.children["Queen"].children["Bet"].children["Fold"],
    win
)

In [ ]:
draw_tree(gbt_one_card_poker, color_scheme="gambit")

Create the game in OpenSpiel:

In [ ]:
ops_one_card_poker = pyspiel.load_efg_game(gbt_one_card_poker.to_efg())
ops_one_card_poker

Games loaded from EFG in OpenSpiel do not take advantage of the full functionality of the package, for example, it is not possible to carry out training with RL algorithms on these games, as in the example above with Tiny Hanabi. The OpenSpiel documentation explains [how to submit new games to the library](https://openspiel.readthedocs.io/en/latest/developer_guide.html#adding-a-game) if you wish to add your own games.

We can however use the state representation and play through the game step by step:

In [ ]:
ops_one_card_poker.num_distinct_actions()

The one-card poker game has 4 distinct actions, 2 are for the first player (Alice in the example game): "Bet" and "Fold", and 2 for the second player (Bob): "Call" and "Fold".

Initialising the game state, we can see the current player at the start is the chance player, who deals the cards:

In [ ]:
state = ops_one_card_poker.new_initial_state()
state

In [ ]:
state.legal_actions()

Let's make the chance player's action dealing a King (action 0):

In [ ]:
state.apply_action(0)
state

In [ ]:
state.legal_actions()

As expected, it's now the first player's (Alice's) turn.
Let's have Alice choose to "Bet" (action 0):

In [ ]:
state.apply_action(0)
state

As expected, the current player is now player 2 (Bob), let's check the legal actions available to Bob:

In [ ]:
state.legal_actions()

Player 2 (Bob) now has the option to "Call" (action 1) or "Fold" (action 2).
Let's have Bob choose to "Fold":

In [ ]:
state.apply_action(2)
state

Since Bob Folded, Alice takes the small win and we reach a terminal state.